In [1]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu
from copy import copy
import statistics as stats
import math
from ta.volume import MFIIndicator
from ta.volatility import AverageTrueRange

In [2]:
# Read recipe inputs
yfinance_ndx_hist_1985_daily = dataiku.Dataset("yfinance_ndx_hist_1985_daily")
df_source = yfinance_ndx_hist_1985_daily.get_dataframe()

In [3]:
# Trend indicator, équivalent de l'affichage couleur
def trend_indicator(trend):
    if trend > 0 :
        # Uptrend
        x = 1
    elif trend < 0 :
        # Downtrend
        x = -1
    else :
        # Range
        x = 0
    return x

In [14]:
# Defintion fonction
def generate_alphatrend(df_in, mfi_seuil, mfi_seuil_entry, atr_l, m):
    '''Paramètres d'entrée : longueur MFI, longueur ATR, multiplier
    Retourne les colonnes Alphatrend, Alphatrend +2, signaux achat/vente
    :mfi_seuil = période MFI servant à délimiter up/down de l'alphatrend
    :mfi_trigger = période MFI pour recherche crossover, leading entry'''

    df = df_in.copy()

    # Colonnes MFI
    s_mfi = MFIIndicator(high=df.High, low=df.Low, close=df.Close, volume=df.Volume, window=mfi_seuil).money_flow_index()
    df["MFI_ref"] = s_mfi

    # Colonne ATR
    s_atr = AverageTrueRange(high=df.High, low=df.Low, close=df.Close, window=atr_l).average_true_range()
    df["ATR"] = s_atr

    # Lignes UpT et DownT
    df["UpT_limit"] = df["Low"] - df["ATR"] * m
    df["DownT_support"] = df["High"] + df["ATR"] * m

    # Suppression des lignes sans signal, en début de DataFrame
    df.dropna(inplace=True)
    df.reset_index(drop=True, inplace=True)

    # ===============================================
    # Calcul Alphatrend, en tant que série
    Alphatrend = [0]

    for i in range (1, df.shape[0]):
        # Cas Uptrend
        if df.at[i,"MFI_ref"] >= mfi_seuil_entry :
            if df.at[i,"UpT_limit"] < Alphatrend[-1] :
                # Flat
                Alphatrend.append(Alphatrend[-1])
            else :
                # Trailing stop loss Up
                Alphatrend.append(df.at[i,"UpT_limit"])

        # Cas Downtrend, MFI < 50
        else :
            if df.at[i,"DownT_support"] > Alphatrend[-1] :
                # Flat
                Alphatrend.append(Alphatrend[-1])
            else :
                # Trailing stop loss Down
                Alphatrend.append(df.at[i,"DownT_support"])

    # ===============================================
    # Ajout des lignes k1 et k2 en tant que colonnes
    if df.shape[0] == len(Alphatrend):
        df["Alphatrend_k1"] = Alphatrend
        # Ligne k2 décalée de 2j
        Alphatrend2 = df["Alphatrend_k1"].shift(periods=2, fill_value=0)
        df["Alphatrend_k2"] = Alphatrend2
        # Trend
        df["Trend"] = df.Alphatrend_k1 - df.Alphatrend_k2
        df["Trend"] = df["Trend"].apply(trend_indicator)
    else :
        print("Erreur lors de la génération des lignes Alphatrend")

    # ===============================================
    # Génération des signaux d'achat/vente
    signal = [np.nan]

    for t in range (1, df.shape[0]):
        if (df.at[t,"Alphatrend_k1"] > df.at[t,"Alphatrend_k2"]) and (df.at[t-1,"Alphatrend_k1"] <= df.at[t-1,"Alphatrend_k2"]):
            # Buy Signal
            signal.append(1)
        elif (df.at[t,"Alphatrend_k1"] < df.at[t,"Alphatrend_k2"]) and (df.at[t-1,"Alphatrend_k1"] >= df.at[t-1,"Alphatrend_k2"]):
            # Sell Signal
            signal.append(-1)
        else :
            # No Signal
            signal.append(0)

    # Ajout des signaux en tant que colone
    if len(signal) == df.shape[0]:
        df["Signal"] = signal
    else :
        print("erreur lors de la génération des signaux achat / vente.")

    # Distance relative du prix de cloture par rapport à k1
    df["dCk1"] = round( (df["Close"] - df["Alphatrend_k1"]) / df["Close"], 2)
    
    
    # Spécification des colonnes avec les paramètres d'entrée
    col_ref = "MFIp" + str(mfi_seuil) + "s" + str(mfi_seuil_entry) + "_ATR" + str(atr_l) + "_m" + str(m)
    
    #df.drop(columns=["Open","High","Low","Close","Volume","MFI_ref","ATR","UpT_limit","DownT_support"], inplace=True)
    df = df[["Date","dCk1","Trend","Signal"]]
    df.columns=["Date", "dCk1_"+col_ref, "trend_"+col_ref, "signal_"+col_ref, ]
    
    return df

In [15]:
#Batterie de valeurs à générer
d_AT_params = {
    "MFI_p" : [12, 14, 14, 14, 17, 17],
    "MFI_s" : [54, 50, 50, 50, 54, 38],
    "ATR" : [14, 14, 14, 14, 14, 14],
    "mult" : [1, 1, 0.75, 1.25, 1, 1]
}

In [21]:
df_alphatrends = df_source[["Date"]]

for e in range (0, len(d_AT_params["MFI_p"]) ):
    df_temp = generate_alphatrend(
        df_source, d_AT_params["MFI_p"][e], 
        d_AT_params["MFI_s"][e], 
        d_AT_params["ATR"][e], 
        d_AT_params["mult"][e]
    )
    df_alphatrends = df_alphatrends.merge(df_temp, how="left", on="Date")

In [30]:
df_alphatrends.dropna(inplace=True)
df_alphatrends.reset_index(drop=True, inplace=True)

In [34]:
# Write recipe outputs
y1985_AT = dataiku.Dataset("y1985_AT")
y1985_AT.write_with_schema(df_alphatrends)

9238 rows successfully written (7ZJ7X7Bo07)
